# HW5 — Bigfoot reports

## Read data and preparation

In [3]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

DATA_URL = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"

df = pd.read_csv(DATA_URL, parse_dates=['date'], low_memory=False)
df = df[df['state'].notna()]
df['Year'] = df['date'].dt.to_period('Y').dt.to_timestamp()

## Counts by year/state

In [4]:
year_state = (
    df.groupby(['Year','state'])
      .size()
      .reset_index(name='reports')
)

## Top states overall

In [5]:
top_states = (year_state.groupby('state')['reports'].sum()
              .nlargest(8).index.tolist())
df_top = year_state[year_state['state'].isin(top_states)]

## Plot 1 (interactive multiline) 

In [6]:
countries = sorted(top_states)
sel = alt.selection_point(
    fields=['state'],
    bind=alt.binding_select(options=sorted(top_states), name='State: '),
    value=sorted(top_states)[0]
)

chart1 = (
    alt.Chart(df_top, title='Bigfoot reports by state over time (top 8)')
      .mark_line()
      .encode(
          x='Year:T',
          y=alt.Y('reports:Q', title='Reports', scale=alt.Scale(type='symlog')),
          color=alt.Color('state:N', legend=None, scale=alt.Scale(scheme='sinebow')),
          size=alt.condition(sel, alt.value(4), alt.value(1)),   # required "size" value
          opacity=alt.condition(sel, alt.value(1), alt.value(0.35)),
          tooltip=['state:N','Year:T','reports:Q']
      )
      .add_params(sel)
      .properties(width=720, height=380)
)

## Plot 2 (static bar) 

In [7]:
totals = (df.groupby('state')
            .size()
            .reset_index(name='reports')
            .sort_values('reports', ascending=False)
            .head(15))

chart2 = (
    alt.Chart(totals, title='Top 15 states by total Bigfoot reports')
      .mark_bar()
      .encode(
          x=alt.X('reports:Q', title='Reports'),
          y=alt.Y('state:N', sort='-x', title='State'),
          color=alt.value('#6baed6'),
          tooltip=['state:N','reports:Q']
      )
      .properties(width=720, height=420)
)

## Save vega-lite JSON for Jekyll embedding (Commit these files under /assets/vegaplots/)

In [11]:
import os
os.makedirs("assets/vegaplots", exist_ok=True)
chart1.save("assets/vegaplots/bigfoot_lines.json")
chart2.save("assets/vegaplots/bigfoot_bars.json")

## Show the two graphs

In [8]:
chart1 | chart2

alt.HConcatChart(...)